In [2]:
### ALL NECESSARY LIBRARIES ###
import pandas as pd
import warnings
import json
import random
warnings.simplefilter(action='ignore')

# for saving variables
import pickle
import os
import tiktoken

# needed for gpt
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

%run "./utilityFunctions.ipynb"

In [52]:
syntheticSynopsisAndSentiment = pd.read_csv("syntheticSynopsisAndSentiment.csv")
syntheticSynopsisAndSentiment

,Synopsis,Sentiment,PHQ8_Score,ParticipantID
0,"The patient grew up in Denver, Colorado and re...",Positive emotions are evident regarding apprec...,9,300
1,"The patient is from Seattle, Washington but re...",Positive sentiments arise regarding love for M...,4,300
2,"The patient originally hails from Seattle, Was...",Positive emotions emerge infrequently related ...,21,300
3,"The patient, originally from Chicago and curre...",Patient shows mixed emotions: generally conten...,6,301
4,"The patient, originally from Chicago and curre...",Patient demonstrates mild depression: generall...,7,301
...,...,...,...,...
538,The patient discusses the challenge of adjusti...,Emotions expressed include excitement (new cit...,3,491
539,The individual recounts feeling burdened by a ...,Emotions expressed involve significant strain ...,13,491
540,The patient recounted their move from Texas to...,Dominated negative emotions: intense dissatisf...,22,492
541,The patient shared their journey moving from S...,Positive emotions: hopefulness (new career pat...,7,492


In [54]:
# Initialize conversation history with the detailed system message
conversation_history = [
]

In [56]:
transcript_prompt_template = """
Your task is to generate a comma-separated list of emotions based on the input sentiments provided below:

{Sentiment}

Output the result in a compact JSON format on a single line without whitespaces. For example:
"Emotions":"positive,pathetic,depressed"
"""



In [57]:
# Pricing details
PROMPT_COST_PER_1000_TOKENS = 0.005
COMPLETION_COST_PER_1000_TOKENS = 0.015

# Initialize the AzureOpenAI client with the specified endpoint and API key
client = AzureOpenAI(
    azure_endpoint="https://gptshuhaotest.openai.azure.com/",  # your Azure endpoint
    api_key="d0a5a4feefc34f71b685fc394d033b2c",  # your API key
    api_version="2024-02-01"
)

MODEL = "gpt-4o"  # change model here

# # Initialize conversation history with a system message
# conversation_history = [
#     {"role": "system", "content": "You are an expert on Animals"}
# ]

# keep track of conversation
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})
    
# clear conversation  
def clear_conversation():
    global conversation_history
    conversation_history = [
    ]


# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_response(prompt):
    # Add the user's message to the conversation history.
    add_message("user", prompt)
    
    # Create a chat completion request to the OpenAI API.
    response = client.chat.completions.create(
        model=MODEL,     
        # The conversation history, including system, user, and assistant messages.
        messages=conversation_history, 
        temperature=0,  # Controls the randomness of the output. 
        max_tokens=2000,  # The maximum number of tokens to generate in the response.
        top_p=1,  # Controls the diversity via nucleus sampling; 1 means use all tokens.
        frequency_penalty=2,  # Penalizes new tokens based on their frequency in the text so far.
        presence_penalty=2,  # Penalizes new tokens based on whether they appear in the text so far.
        n=1,  # The number of completions to generate. Here, we are generating only one.
        response_format={ "type": "json_object" },
    )

    # Add the assistant's message to the conversation history.
    assistant_message = response.choices[0].message.content
    add_message("assistant", assistant_message)
    
    
    # Save usage and compute cost
    usage = response.usage
    computeCost(usage)

    # Return the assistant's message.
    return assistant_message

In [60]:
Emotions = []

for index, row in syntheticSynopsisAndSentiment.iterrows():
    prompt_multiplier = 1
    

    for _ in range(prompt_multiplier):
        attempt_count = 0
        success = False
        while attempt_count < 10 and not success:
            attempt_count += 1
            # Format the prompt with the current row's conversation
            prompt = transcript_prompt_template.format(
                Sentiment = row["Sentiment"]
            )
            try:
                # Get synthetic data response
                response = get_response(prompt)
                print(f"Participant ID {row['ParticipantID']}")
                print(f"Attempt {attempt_count}: {response}")  # More contextual printing

                # Try to parse the response as JSON
                json_data = json.loads(response)

                # Check if the required keys are present in the JSON data
                if 'Emotions' in json_data:
                    json_data['ParticipantID'] = row['ParticipantID']  # Add ParticipantID to JSON data
                    json_data['PHQ8_Score'] = row['PHQ8_Score'] 
                    json_data['Sentiment'] = row['Sentiment'] 
                    json_data['Synopsis'] = row['Synopsis'] 
                    Emotions.append(json_data)
                    success = True
                else:
                    print("Error: Required keys are missing from the JSON data. Retrying...")
                    clear_conversation()
                    continue  # Continue to retry since the JSON does not have the required keys

            except json.JSONDecodeError:
                # If JSON parsing fails, retry fetching the response
                print(f"Error: Response not loadable as JSON (Attempt {attempt_count}). Retrying...")
                clear_conversation()  # Reset conversation if needed for next attempt

            except Exception as e:
                # Handle any other exceptions, including BadRequestError
                print(f"Error encountered: {e}. Skipping this prompt.")
                clear_conversation()  # Exit the while loop and skip to the next row

        # Clear conversation context if necessary, post all attempts
        clear_conversation()

# Convert the list of JSON objects to a DataFrame
Emotions_df = pd.DataFrame(Emotions)


Emotions_df.to_csv('syntheticSynopsisAndSentiment.csv', index=False)

Emotions_df 